# Barrier Option Pricing



In [73]:
# ------------------------------------------------------------------------------
# Published in:     Comp_FiMa_class_WS2021
# ------------------------------------------------------------------------------
# Project title:    Barrier Option Pricing and Sensitivity
# ------------------------------------------------------------------------------
# Description:      Calculate the price of barrier option as well as its sensitivity
# ------------------------------------------------------------------------------
# Required Packages:  numpy, py_vollib, scipy.stats
# ------------------------------------------------------------------------------
# Author :          Ananda Eraz Irfananto (547048)
# ------------------------------------------------------------------------------



In [89]:
pip install py_vollib

Note: you may need to restart the kernel to use updated packages.


In [90]:
import numpy as np

In [91]:
# S0  Basiswert
# K   Strike-Preis
# T   Maturität im Jahr (betrachtende Laufzeit)
# H   Barrierpreis
# r   Zins
# N   Anzahl von Schritt
# u   up-faktor 
# opt Optiontyp Call oder Put

def barrieropt(S0,K,T,H,r,N,u,opt):
    #Konstante definieren
    dt = T/N  # Länge
    d = 1/u  # down-faktor
    q = (np.exp(r * dt) - d)/(u - d) # Erfolgswahrscheinlichkeit (risikoneutrales Maß)
    disk = np.exp(-r * dt) # Diskontierung
    
    
    #Wert von Underlyings am Maturität
    S = np.zeros(N+1)
    for j in range(0, N+1):
        S[j] = S0 * u**j * d**(N - j)
    
    #Option payoff
    C = np.zeros(N+1)
    for j in range(0, N+1):
        if opt == 'C':
            C[j] = max(0, S[j] - K)
        elif opt == 'P':
            C[j] = max(0, K - S[j])
            
    #Barrier prüfen
    for j in range(0, N+1):
        S = S0 * u**j * d**(N - j)
        if S >= H:
            C[j] = 0
        
            
    #Rückwärts durch den Binomialbaum
    for i in np.arange(N-1, -1, -1):
        for j in range(0, i+1):
            S = S0 * u**j * d**(i - j)
            if S >= H:
                C[j] = 0
            else:
                C[j] = disk * (q * C[j+1] + (1 - q) * C[j])
    return C[0]


        

In [92]:
# S0 = 100  Basiswert
# K = 100   Strike-Preis
# T = 1     Maturität im Jahr (betrachtende Laufzeit)
# H = 125   up-and-out Barrierpreis
# r = 0.06  Zins
# N = 3     Anzahl von Schritt
# u = 1.1   up-faktor 
barrieropt(100,100,1,125,0.06,3,1.1,opt='C')  # Call-Option

4.00026736854323

In [93]:
# S0 = 200  Basiswert
# K = 250   Strike-Preis
# T = 1     Maturität im Jahr (betrachtende Laufzeit)
# H = 80    down-and-out Barrierpreis
# r = 0.06  Zins
# N = 3     Anzahl von Schritt
# u = 1.1   up-faktor 
barrieropt(200,250,1,80,0.06,3,1.1,opt='C')  # Call-Option

0.0

In [94]:
# S0 = 100  Basiswert
# K = 100   Strike-Preis
# T = 1     Maturität im Jahr (betrachtende Laufzeit)
# H = 125   up-and-out Barrierpreis
# r = 0.06  Zins
# N = 3     Anzahl von Schritt
# u = 1.1   up-faktor 
barrieropt(100,100,1,125,0.06,3,1.1,opt='P')  # Put-Option

4.322189158353709

In [95]:
# S0 = 200  Basiswert
# K = 250   Strike-Preis
# T = 1     Maturität im Jahr (betrachtende Laufzeit)
# H = 80    down-and-out Barrierpreis
# r = 0.06  Zins
# N = 3     Anzahl von Schritt
# u = 1.1   up-faktor
barrieropt(200,250,1,80,0.06,3,1.1,opt='P')  # Put-Option

0.0

# Sensitivität


In [96]:
from scipy.stats import norm
from py_vollib.black_scholes import black_scholes as bs
from py_vollib.black_scholes.greeks.analytical import delta, gamma, vega, theta, rho


In [97]:
# Variablen definieren
r = 0.06 # Zins
S = 100   # Optionswert
K = 100   # Strike-Preis
T = 240/365 # Laufzeit
sigma = 0.30 # Volatilität

def blackScholes(r, S, K, T, sigma, type="c"):
    # Black-Scholes Preis berechnen
    d1 = (np.log(S/K) + (r + sigma**2/2)*T)/(sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    try:
        if type == "c":
            preis = S*norm.cdf(d1, 0, 1) - K*np.exp(-r*T)*norm.cdf(d2, 0, 1)
        elif type == "p":
            preis = K*np.exp(-r*T)*norm.cdf(-d2, 0, 1) - S*norm.cdf(-d1, 0, 1)
        return preis, bs(type, S, K, T, r, sigma)
    except:
        print("Bitte geben Sie 'type' ein")


def delta_calc(r, S, K, T, sigma, type="c"):
    # Delta berechnen
    d1 = (np.log(S/K) + (r + sigma**2/2)*T)/(sigma*np.sqrt(T))
    try:
        if type == "c":
            delta_calc = norm.cdf(d1, 0, 1)
        elif type == "p":
            delta_calc = -norm.cdf(-d1, 0, 1)
        return delta_calc, delta(type, S, K, T, r, sigma)
    except:
        print("Bitte geben Sie 'type' ein")

        
def gamma_calc(r, S, K, T, sigma, type="c"):
    # Gamma berechnen
    d1 = (np.log(S/K) + (r + sigma**2/2)*T)/(sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    try:
        gamma_calc = norm.pdf(d1, 0, 1)/(S*sigma*np.sqrt(T))
        return gamma_calc, gamma(type, S, K, T, r, sigma)
    except:
        print("Bitte geben Sie 'type' ein")


def vega_calc(r, S, K, T, sigma, type="c"):
    # Vega berechnen
    d1 = (np.log(S/K) + (r + sigma**2/2)*T)/(sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    try:
        vega_calc = S*norm.pdf(d1, 0, 1)*np.sqrt(T)
        return vega_calc*0.01, vega(type, S, K, T, r, sigma)
    except:
        print("Bitte geben Sie 'type' ein")
        
def theta_calc(r, S, K, T, sigma, type="c"):
    # Theta berechnen
    d1 = (np.log(S/K) + (r + sigma**2/2)*T)/(sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    try:
        if type == "c":
            theta_calc = -S*norm.pdf(d1, 0, 1)*sigma/(2*np.sqrt(T)) - r*K*np.exp(-r*T)*norm.cdf(d2, 0, 1)
        elif type == "p":
            theta_calc = -S*norm.pdf(d1, 0, 1)*sigma/(2*np.sqrt(T)) + r*K*np.exp(-r*T)*norm.cdf(-d2, 0, 1)
        return theta_calc/365, theta(type, S, K, T, r, sigma)
    except:
        print("Bitte geben Sie 'type' ein")


def rho_calc(r, S, K, T, sigma, type="c"):
    # Rho berechnen
    d1 = (np.log(S/K) + (r + sigma**2/2)*T)/(sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    try:
        if type == "c":
            rho_calc = K*T*np.exp(-r*T)*norm.cdf(d2, 0, 1)
        elif type == "p":
            rho_calc = -K*T*np.exp(-r*T)*norm.cdf(-d2, 0, 1)
        return rho_calc*0.01, rho(type, S, K, T, r, sigma)
    except:
        print("Bitte geben Sie 'type' ein")



In [98]:
option_type='p'

print("Option Preis: ", [round(x,3) for x in blackScholes(r, S, K, T, sigma, option_type)])
print("       Delta: ", [round(x,3) for x in delta_calc(r, S, K, T, sigma, option_type)])
print("       Gamma: ", [round(x,3) for x in gamma_calc(r, S, K, T, sigma, option_type)])
print("       Vega : ", [round(x,3) for x in vega_calc(r, S, K, T, sigma, option_type)])
print("       Theta: ", [round(x,3) for x in theta_calc(r, S, K, T, sigma, option_type)])
print("       Rho  : ", [round(x,3) for x in rho_calc(r, S, K, T, sigma, option_type)])

Option Preis:  [7.684, 7.684]
       Delta:  [-0.388, -0.388]
       Gamma:  [0.016, 0.016]
       Vega :  [0.311, 0.311]
       Theta:  [-0.012, -0.012]
       Rho  :  [-0.306, -0.306]


In [99]:
option_type='c'

print("Option Preis: ", [round(x,3) for x in blackScholes(r, S, K, T, sigma, option_type)])
print("       Delta: ", [round(x,3) for x in delta_calc(r, S, K, T, sigma, option_type)])
print("       Gamma: ", [round(x,3) for x in gamma_calc(r, S, K, T, sigma, option_type)])
print("       Vega : ", [round(x,3) for x in vega_calc(r, S, K, T, sigma, option_type)])
print("       Theta: ", [round(x,3) for x in theta_calc(r, S, K, T, sigma, option_type)])
print("       Rho  : ", [round(x,3) for x in rho_calc(r, S, K, T, sigma, option_type)])

Option Preis:  [11.552, 11.552]
       Delta:  [0.612, 0.612]
       Gamma:  [0.016, 0.016]
       Vega :  [0.311, 0.311]
       Theta:  [-0.028, -0.028]
       Rho  :  [0.326, 0.326]
